Code to develop tools for calculating p-values from streaming bootstrap data

In [1]:
%load_ext autoreload
%autoreload 2

In [74]:
import numpy as np

from janelia_core.stats.bootstrap import OnlineNonZeroPVlCalculator
from janelia_core.stats.regression import grouped_linear_regression_boot_strap_stats

## Parameters go here

In [80]:
var_mns = [0, 3, -3]
var_stds = [1, 2, 3]
n_data_smps = 1000
n_bs_smps = 1000

# Generate the original data

In [81]:
n_vars = len(var_mns)
data = np.random.randn(n_data_smps, n_vars)*var_stds + var_mns

## Generate the boostrap estimates of the mean

In [82]:
bs_mn_estimates = np.zeros([n_bs_smps, n_vars])
for b_i in range(n_bs_smps):
    bs_inds = np.random.choice(n_data_smps, n_data_smps, replace=True)
    bs_mn_estimates[b_i, :] = np.mean(data[bs_inds, :], axis=0)

## Use streaming code to calculate p-values that the means of each random variable are different than 0

In [83]:
ol_p_vl_calc = OnlineNonZeroPVlCalculator(n_vars)
for b_i in range(n_bs_smps):
    ol_p_vl_calc.injest(bs_mn_estimates[np.newaxis, b_i,:])
ol_p_vl_calc.p_vls()

array([0.222, 0.002, 0.002])

## Use batch code to calculate the same p-values

In [84]:
stats = grouped_linear_regression_boot_strap_stats(bs_mn_estimates)

In [85]:
stats['non_zero_p']

array([0.222, 0.002, 0.002])